### Токенизация

Токенизация: деление на т.н. "токены" &ndash; какие-либо значимые элементы:

1. Предложения
2. Слова
3. Морфемы
4. Графемы
5. ...

По каким признакам можно делить на предложения? Какие могут быть проблемы?

In [4]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [27]:
text = 'Был тихий серый вечер. "Кто это?", спросил мальчик, заглядывая на следующую страницу.'

sentences = nltk.tokenize.sent_tokenize(text, "russian")
print(sentences)

['Был тихий серый вечер.', '"Кто это?', '", спросил мальчик, заглядывая на следующую страницу.']


https://aclanthology.org/J06-4003.pdf

Какие могут возникнуть проблемы при делении на слова?

In [6]:
for sent in sentences:
    print(nltk.tokenize.word_tokenize(sent, "russian"))

['Был', 'тихий', 'серый', 'вечер', '.']
['``', 'Кто', 'это', '?']
['``', ',', 'спросил', 'мальчик', ',', 'заглядывая', 'на', 'следующую', 'страницу', '.']


Токенизация с помощью регулярных выражений:

In [29]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r"\w+")
tokenizer.tokenize(text)

['Был',
 'тихий',
 'серый',
 'вечер',
 'Кто',
 'это',
 'спросил',
 'мальчик',
 'заглядывая',
 'на',
 'следующую',
 'страницу']

### Стемминг

In [7]:
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer("russian")

text = "был тихий серый вечер дул ветер слабый и тёплый небо было покрыто тучами"
for word in text.split():
    print(stemmer.stem(word))

был
тих
сер
вечер
дул
ветер
слаб
и
тепл
неб
был
покрыт
туч


#### Проблемы стемминга:

In [8]:
# чередование в основах
print(stemmer.stem("ветер"))
print(stemmer.stem("ветра"))

ветер
ветр


In [9]:
# омонимия основ
print(stemmer.stem("банком"))
print(stemmer.stem("банкой"))

банк
банк


In [10]:
# супплетивизм
print(stemmer.stem("человек"))
print(stemmer.stem("люди"))

человек
люд


### Лемматизация

In [ ]:
!pip install pymorphy3

In [1]:
import pymorphy3 as pymorphy
morph = pymorphy.MorphAnalyzer()

In [11]:
for word in text.split():
    print(morph.parse(word)[0].normal_form)

быть
тихий
серый
вечер
дуть
ветер
слабый
и
тёплый
небо
быть
покрыть
туча


Проверим проблемные места:

In [12]:
print(morph.parse("ветер")[0].normal_form)
print(morph.parse("ветра")[0].normal_form)

ветер
ветер


In [13]:
print(morph.parse("банком")[0].normal_form)
print(morph.parse("банкой")[0].normal_form)

банк
банка


In [14]:
print(morph.parse("человек")[0].normal_form)
print(morph.parse("люди")[0].normal_form)

человек
человек


### Морфологический анализ

https://pymorphy2.readthedocs.io/en/stable/user/grammemes.html

В pymoprhy используются граммемы OpenCorpora с небольшими изменениями:

https://opencorpora.org/dict.php?act=gram

In [ ]:
for word in text.split():
    print(morph.parse(word)[0])

In [ ]:
parsres = morph.parse("тихий")
parsres

[Parse(word='тихий', tag=OpencorporaTag('ADJF,Qual inan,masc,sing,accs'), normal_form='тихий', score=0.666666, methods_stack=((DictionaryAnalyzer(), 'тихий', 738, 4),)),
 Parse(word='тихий', tag=OpencorporaTag('ADJF,Qual masc,sing,nomn'), normal_form='тихий', score=0.333333, methods_stack=((DictionaryAnalyzer(), 'тихий', 738, 0),))]

In [ ]:
parsres = morph.parse("курдячит")
parsres

[Parse(word='курдячит', tag=OpencorporaTag('VERB,impf,intr sing,3per,pres,indc'), normal_form='курдячать', score=0.5384615384615384, methods_stack=((DictionaryAnalyzer(), 'ячит', 564, 5), (UnknownPrefixAnalyzer(score_multiplier=0.5), 'курд'))),
 Parse(word='курдячит', tag=OpencorporaTag('VERB,impf,intr sing,3per,pres,indc'), normal_form='курдячить', score=0.23076923076923075, methods_stack=((FakeDictionary(), 'курдячит', 371, 5), (KnownSuffixAnalyzer(min_word_length=4, score_multiplier=0.5), 'ячит'))),
 Parse(word='курдячит', tag=OpencorporaTag('VERB,perf,intr sing,3per,futr,indc'), normal_form='курдячить', score=0.23076923076923075, methods_stack=((FakeDictionary(), 'курдячит', 1551, 9), (KnownSuffixAnalyzer(min_word_length=4, score_multiplier=0.5), 'ячит')))]

In [ ]:
[i for i in dir(parsres[0]) if not i.startswith("_")]

['count',
 'index',
 'inflect',
 'is_known',
 'lexeme',
 'make_agree_with_number',
 'methods_stack',
 'normal_form',
 'normalized',
 'score',
 'tag',
 'word']

In [ ]:
parsres[0].lexeme

In [ ]:
parsres[0].inflect({"femn", "accs"})

Parse(word='тихую', tag=OpencorporaTag('ADJF,Qual femn,sing,accs'), normal_form='тихий', score=1.0, methods_stack=((DictionaryAnalyzer(), 'тихую', 738, 10),))

In [49]:
tag = morph.parse("тихий")[0].tag
print(tag)

ADJF,Qual inan,masc,sing,accs


In [50]:
dir(tag)

['ANIMACY',
 'ASPECTS',
 'CASES',
 'FORMAT',
 'GENDERS',
 'INVOLVEMENT',
 'KNOWN_GRAMMEMES',
 'MOODS',
 'NUMBERS',
 'PARTS_OF_SPEECH',
 'PERSONS',
 'POS',
 'RARE_CASES',
 'TENSES',
 'TRANSITIVITY',
 'VOICES',
 '_CYR2LAT',
 '_EXTRA_INCOMPATIBLE',
 '_GRAMMEME_INCOMPATIBLE',
 '_GRAMMEME_INDICES',
 '_LAT2CYR',
 '_NON_PRODUCTIVE_GRAMMEMES',
 '_NUMERAL_AGREEMENT_GRAMMEMES',
 '_POS',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__firstlineno__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__static_attributes__',
 '__str__',
 '__subclasshook__',
 '_assert_grammemes_are_known',
 '_assert_grammemes_initialized',
 '_cyr',
 '_cyr_grammemes_cache',
 '_from_internal_grammeme',
 '_from_internal_tag',
 '_grammemes_cache',
 

In [51]:
print(tag.POS, tag.number, tag.case, tag.gender)

ADJF sing accs masc


In [52]:
"ADJF" in tag

True

In [53]:
"NOUN" in tag

False

In [55]:
{"accs", "sing"} in tag

True

In [57]:
morph.parse(".")[0].tag

OpencorporaTag('PNCT')

### Задания для самостоятельного выполнения

#### Задание 1: токенизация на графемы

Даны два текста на русском языке в транскрипции с разными обозначениями.

* https://pkholyavin.github.io/year4programming/sibling_sample.txt
* https://pkholyavin.github.io/year4programming/corpres_sample.txt

1. Для каждого текста внимательно изучить принятые обозначения, определить, в каком случае возможна однозначная токенизация, а в каком невозможна, и почему.
2. Для текста, в котором она возможна, написать программу, которая каждую строчку превращает в список звуков. Бонусные очки, если сделаете с помощью регулярных выражений.

In [ ]:
!wget -q https://phonetics-spbu.github.io/courses/ling_programming/files/sibling_sample.txt
!wget -q https://phonetics-spbu.github.io/courses/ling_programming/files/corpres_sample.txt

#### Задание 2 + домашнее задание

1. Возьмите любой текст (например, новостной или скачать с wikisource или lib.ru)
2. С помощью nltk разбейте текст на предложения, каждое предложение разбить на слова.
3. Постройте словари слов: алфавитный список, частотный словарь, частотный словарь лексем.
4. Постройте алфавитный список всех глаголов. Каждый глагол поставьте в форму первого лица, единственного числа и настоящего времени, если глагол несовершенного вида, и будущего, если он совершенного вида.